In [1]:
# conda activate {env} doesn't work well here
# so we manually modify the path
PATH=$CONDA_PREFIX/envs/ploomber/bin:$PATH

# Running experiments in parallel

- explain the top raw cell with config and how to add it
- explain the parameters cell

In [9]:
ploomber cloud nb notebooks/grid.ipynb

Uploading grid-7bd167c1.ipynb...
Triggering execution of grid-7bd167c1.ipynb...


Check that the task was submitted:

In [10]:
ploomber cloud list

created_at      runid                                 status
--------------  ------------------------------------  --------
20 seconds ago  f858f6b3-9b3f-4c86-b55d-de23fcba045f  created
an hour ago     2e0bae66-6c90-4ad5-84a9-b6cdb746a5af  finished
10 hours ago    0dd11c31-3200-41cc-81ca-ff0f59bc03e7  finished
11 hours ago    32e54117-4890-4001-93a3-beacc394e983  finished
11 hours ago    3e30c17f-f789-4771-8b6a-7bf34d89d63e  finished


Wait for 1-2 minutes for the Docker image to build, you'll see this once it's done:

In [16]:
ploomber cloud logs @latest --image | tail -n 10

[Container] 2022/10/21 13:26:33 Phase complete: BUILD State: SUCCEEDED

[Container] 2022/10/21 13:26:33 Phase context status code:  Message: 

[Container] 2022/10/21 13:26:33 Entering phase POST_BUILD

[Container] 2022/10/21 13:26:33 Phase complete: POST_BUILD State: SUCCEEDED

[Container] 2022/10/21 13:26:33 Phase context status code:  Message: 



Now you'll see that the notebook has `started`:

In [17]:
ploomber cloud list

created_at     runid                                 status
-------------  ------------------------------------  --------
3 minutes ago  f858f6b3-9b3f-4c86-b55d-de23fcba045f  started
an hour ago    2e0bae66-6c90-4ad5-84a9-b6cdb746a5af  finished
11 hours ago   0dd11c31-3200-41cc-81ca-ff0f59bc03e7  finished
11 hours ago   32e54117-4890-4001-93a3-beacc394e983  finished
11 hours ago   3e30c17f-f789-4771-8b6a-7bf34d89d63e  finished


Let's see the status of each individual task:

In [19]:
ploomber cloud status @latest

Geting latest ID...
Got ID: f858f6b3-9b3f-4c86-b55d-de23fcba045f
Unknown status: started
taskid                     name             runid                      status
-------------------------  ---------------  -------------------------  --------
59f28ad3-1db2-494d-8de9-d  grid-7bd167c1-2  f858f6b3-9b3f-4c86-b55d-d  created
33c9e6306f8                                 e23fcba045f
7a318b97-6de8-4431-9f7c-6  grid-7bd167c1-0  f858f6b3-9b3f-4c86-b55d-d  created
abd6c454e96                                 e23fcba045f
8d07cc57-3fd7-44eb-a761-5  grid-7bd167c1-3  f858f6b3-9b3f-4c86-b55d-d  created
50ab3bacd28                                 e23fcba045f
26098820-1487-4a47-af36-b  grid-7bd167c1-1  f858f6b3-9b3f-4c86-b55d-d  created
179f09055b6                                 e23fcba045f


After a few minutes, they are done:

In [21]:
ploomber cloud status @latest

Geting latest ID...
Got ID: f858f6b3-9b3f-4c86-b55d-de23fcba045f
Pipeline failed...
taskid                     name             runid                      status
-------------------------  ---------------  -------------------------  --------
59f28ad3-1db2-494d-8de9-d  grid-7bd167c1-2  f858f6b3-9b3f-4c86-b55d-d  failed
33c9e6306f8                                 e23fcba045f
7a318b97-6de8-4431-9f7c-6  grid-7bd167c1-0  f858f6b3-9b3f-4c86-b55d-d  failed
abd6c454e96                                 e23fcba045f
8d07cc57-3fd7-44eb-a761-5  grid-7bd167c1-3  f858f6b3-9b3f-4c86-b55d-d  failed
50ab3bacd28                                 e23fcba045f
26098820-1487-4a47-af36-b  grid-7bd167c1-1  f858f6b3-9b3f-4c86-b55d-d  failed
179f09055b6                                 e23fcba045f


In [25]:
# ploomber cloud logs @latest

# Uploading input data

If your notebook requires input data, you can upload it to the cloud. Let's see what happens if we try to run a notebook with missing input data:

In [26]:
ploomber cloud nb notebooks/input-data.ipynb

Uploading input-data-dea2d666.ipynb...
Triggering execution of input-data-dea2d666.ipynb...
Error: Error validating inputs/outputs: {'missing': {'../data/penguins.csv'}} (status: 400)


: 1

Mention that prefix is required here, otherwise Ploomber will add a unique ID:

In [30]:
ploomber cloud nb notebooks/input-data-with-raw-cell.ipynb

Uploading input-data-with-raw-cell-05693b8a.ipynb...
Triggering execution of input-data-with-raw-cell-05693b8a.ipynb...
Error: Cannot start execution. The following inputs are missing:
	- ../data/penguins.csv
Upload them to your data workspace or using the CLI: 
ploomber cloud data --upload ../data/penguins.csv --prefix penguins-classification/input --name data-penguins.csv
 (status: 400)


: 1

Let's get the data:

In [15]:
mkdir -p data
curl https://raw.githubusercontent.com/mwaskom/seaborn-data/master/penguins.csv -o data/penguins.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 13478  100 13478    0     0  98130      0 --:--:-- --:--:-- --:--:-- 98379


Use the command printed above:

In [33]:
# NOTE: you may need to change the path in the --upload argument if
# the file is somewhere else
ploomber cloud data --upload data/penguins.csv \
    --prefix penguins-classification/input \
    --name data-penguins.csv

Uploading data-penguins.csv...


In [40]:
ploomber cloud nb notebooks/input-data-with-raw-cell.ipynb

Uploading input-data-with-raw-cell-6bf5382f.ipynb...
Triggering execution of input-data-with-raw-cell-6bf5382f.ipynb...


Wait for a couple of minutes:

In [48]:
ploomber cloud list

created_at      runid                                 status
--------------  ------------------------------------  --------
4 minutes ago   41fe4e15-fc0f-40bc-b07a-0d45457ba84b  finished
14 minutes ago  ffe17c8c-a5a6-4561-a53a-5e7fd4045036  failed
55 minutes ago  f858f6b3-9b3f-4c86-b55d-de23fcba045f  failed
2 hours ago     2e0bae66-6c90-4ad5-84a9-b6cdb746a5af  finished
11 hours ago    0dd11c31-3200-41cc-81ca-ff0f59bc03e7  finished


Explain that the prefix determines where the outputs are stored:

In [52]:
ploomber cloud download 'penguins-classification/*'

Writing file into path penguins-classification/output/.notebook.ipynb.metadata
Writing file into path penguins-classification/output/notebook.ipynb


# Organizing artifacts

```yaml
prefix: some-experiment
```

# Requesting more memory, CPU and GPU

```yaml
task_resources:
    vcpus: 8 # number of CPUs
    memory: 16384 # memory in MiB
```

`resources.ipynb`

# Specify package versions

```yaml
dependencies:
    - matplotlib==3.5.3
    - scikit-learn==1.1.0
```

`dependencies.ipynb`

# Debugging

- image logs
- task logs
- downloading artifacts